# Reranking for Enhanced RAG Systems
이 노트북에서는 RAG 시스템의 검색 품질을 향상시키기 위한 검색 결과에 대한 리랭킹 기법을 구현합니다.   
리랭킹은 초기 검색 이후 한번 더 필터링을 수행하여, 응답 생성에 가장 적합한 콘텐츠가 사용되도록 합니다.

### 재정렬의 핵심 개념
1. 초기 검색: 기본 유사도 검색을 통해 1차적으로 문서를 빠르게 찾음(정확도는 다소 낮음)
2. 문서 점수화: 검색된 각 문서가 쿼리와 얼마나 관련 있는지 평가
3. Reordering: 문서의 관련성 점수를 기준으로 순서 재배치
4. 선택: 응답 생성에 가장 적합한 문서만 최종적으로 사용

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [1]:
import os
import numpy as np
from dotenv import load_dotenv
import re

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 일부가 겹치도록 chunking 합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 리스트
    
    # overlap만큼 겹치도록 text를 n의 길이로 chunking
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])

    return chunks  

## Simple Vector Store 구축
NumPy를 사용하여 간단한 Vecotr store 구축

In [ ]:
class SimpleVectorStore:
    """
    NumPy를 사용하여 간단한 Vecotr store 구축
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목 추가

        Args:
        text (str): 원본 텍스트.
        embedding (List[float]): 임베딩 벡터.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, k=5):
        """
        시맨틱 서치 수행

        Args:
        query_embedding (List[float]): 쿼리 임베딩 벡터.
        k (int): 반환할 결과의 수.

        Returns:
        List[Dict]: 텍스트와 메타데이터가 포함된 상위 k개 유사 항목.
        """
        if not self.vectors:
            return []
        
        # 쿼리 임베딩을 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 사용하여 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도에 따라 정렬 (내림차순)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })
        
        return results

## 임베딩 생성

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

## 문서 처리 파이프라인
이제 필요한 함수들과 클래스들을 모두 정의했으니, 본격적으로 문서 처리 과정을 구성해보겠습니다.

In [ ]:
def process_document(file_path, chunk_size=1000, chunk_overlap=200):
    """
    RAG를 위한 문서를 전처리합니다.

    Args:
    file_path (str): 파일 경로
    chunk_size (int): 각 청크의 문자 단위 크기
    chunk_overlap (int): 청크 간 겹치는 문자 수

    Returns:
    SimpleVectorStore: 문서 청크와 임베딩이 포함된 벡터 저장소
    """
    # PDF 파일에서 텍스트 추출
    # print("Extracting text from PDF...")
    # extracted_text = extract_text_from_pdf(pdf_path)

    # 텍스트 파일 로드
    extracted_text = load_text_file(file_path)
    
    # 추출된 텍스트 청킹
    print("Chunking text...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"Created {len(chunks)} text chunks")
    
    # 청크 임베딩 생성
    print("Creating embeddings for chunks...")
    chunk_embeddings = create_embeddings(embedding_model, chunks, device=device, batch_size=1)
    
    # 벡터 저장소 생성
    store = SimpleVectorStore()
    
    # 각 청크와 임베딩을 VectorStore에 저장
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": file_path}
        )
    
    print(f"Added {len(chunks)} chunks to the vector store")
    return store

## LLM 기반 Reranking 구현
OpenAI API를 사용하여 LLM 기반 Reranking 기능 구현


In [ ]:
def rerank_with_llm(query, results, top_n=3, model_name="gpt-4.1-mini"):
    """
    LLM 관련성 점수를 사용하여 검색 결과를 reranking합니다.
    
    Args:
        query (str): 사용자 쿼리
        results (List[Dict]): 초기 검색 결과
        top_n (int): reranking 후 반환할 결과의 수
        model_name (str): 점수 계산에 사용할 모델
        
    Returns:
        List[Dict]: 재정렬된 결과
    """
    print(f"Reranking {len(results)} documents...")  # reranking 할 문서 수 출력
    
    scored_results = []  # 점수가 매겨진 결과를 저장할 빈 리스트 초기화
    
    # 시스템 프롬프트
    system_prompt = """You are an expert at evaluating document relevance for search queries.
Your task is to rate documents on a scale from 0 to 10 based on how well they answer the given query.

Guidelines:
- Score 0-2: Document is completely irrelevant
- Score 3-5: Document has some relevant information but doesn't directly answer the query
- Score 6-8: Document is relevant and partially answers the query
- Score 9-10: Document is highly relevant and directly answers the query

You MUST respond with ONLY a single integer score between 0 and 10. Do not include ANY other text."""
    
    # 초기 검색 결과를 Context로 활용해 하나의 prompt 생성
    for i, result in enumerate(results):
        # 진행 상태 출력
        if i % 5 == 0:
            print(f"Scoring document {i+1}/{len(results)}...")
        
        # user prompt
        user_prompt = f"""Query: {query}

Document:
{result['text']}

Rate this document's relevance to the query on a scale from 0 to 10:"""
        
        # LLM으로 각 청크별 점수 평가
        response = client_openai.chat.completions.create(
            model=model_name,
            temperature=0,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        
        # 점수 추출
        score_text = response.choices[0].message.content.strip()
        score_match = re.search(r'\b(10|[0-9])\b', score_text)

        if score_match:
            score = float(score_match.group(1))
        else:
            # 점수 추출 실패 시 유사도 점수 그대로 사용
            print(f"Warning: Could not extract score from response: '{score_text}', using similarity score instead")
            score = result["similarity"] * 10
        
        # 점수가 매겨진 결과를 리스트에 추가
        scored_results.append({
            "text": result["text"],
            "metadata": result["metadata"],
            "similarity": result["similarity"],
            "relevance_score": score
        })
    
    # 유사도(관련성) 점수로 정렬 (내림차순)
    reranked_results = sorted(scored_results, key=lambda x: x["relevance_score"], reverse=True)
    
    # 상위 top_n 결과 반환
    return reranked_results[:top_n]

## 간단한 Keyword-based Reranking

In [ ]:
def rerank_with_keywords(query, results, top_n=3):
    """
    키워드 일치와 위치를 기반으로 하는 간단한 대안 재정렬 방법
    
    Args:
        query (str): 사용자 쿼리
        results (List[Dict]): 초기 검색 결과
        top_n (int): rerank 후 반환할 결과의 수
        
    Returns:
        List[Dict]: Reranked results
    """
    # 쿼리에서 중요한 키워드 추출
    keywords = [word.lower() for word in query.split() if len(word) > 3]
    
    scored_results = []  
    
    for result in results:
        document_text = result["text"].lower()  # 문서 텍스트를 소문자로 변환
        
        # 기본 점수는 벡터 유사도로 시작
        base_score = result["similarity"] * 0.5
        
        # 키워드 점수 초기화
        keyword_score = 0
        for keyword in keywords:
            if keyword in document_text:
                # 찾은 키워드에 대해 점수 추가
                keyword_score += 0.1
                
                # 키워드가 처음 나타나는 위치에 더 많은 점수 추가
                first_position = document_text.find(keyword)
                if first_position < len(document_text) / 4:  # 텍스트의 첫 번째 사분의 위치에 더 많은 점수 추가
                    keyword_score += 0.1
                
                # 키워드 빈도에 대해 점수 추가
                frequency = document_text.count(keyword)
                keyword_score += min(0.05 * frequency, 0.2)  # 최대 0.2로 제한
        
        # 기본 점수와 키워드 점수를 결합하여 최종 점수 계산
        final_score = base_score + keyword_score
        
        # 점수가 매겨진 결과를 리스트에 추가
        scored_results.append({
            "text": result["text"],
            "metadata": result["metadata"],
            "similarity": result["similarity"],
            "relevance_score": final_score
        })
    
    # 최종 관련성 점수로 정렬 (내림차순)
    reranked_results = sorted(scored_results, key=lambda x: x["relevance_score"], reverse=True)
    
    # 상위 top_n 결과 반환
    return reranked_results[:top_n]

## 검색된 청크를 기반으로 response 생성하기

In [ ]:
def generate_response(query, context, model_name='gpt-4.1-nano'):

    # 시스템 프롬프트
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
    
    # user 프롬프트
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        Please answer the question based only on the context provided above. Be concise and accurate.
    """
    # 응답 생성
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content

## Full RAG Pipeline with Reranking
지금까지 문서 처리, 질문 응답, Reranking을 포함한 RAG 파이프라인의 핵심 구성 요소를 구현했습니다.  
이제 이러한 구성 요소를 결합하여 완전한 RAG 파이프라인을 만들어 보겠습니다.

In [ ]:
def rag_with_reranking(query, vector_store, reranking_method="llm", top_n=3, model_name="gpt-4.1-mini"):
    """
    Reranking을 포함한 RAG 파이프라인
    
    Args:
        query (str): 사용자 쿼리
        vector_store (SimpleVectorStore): 벡터 저장소
        reranking_method (str): 재정렬 방법 ('llm' 또는 'keywords')
        top_n (int): rerank 후 반환할 결과의 수
        model (str): Model for response generation
        
    Returns:
        Dict: 쿼리, 컨텍스트, 응답이 포함된 결과
    """
    # 쿼리 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)[0]
    
    # 초기 검색 (top_n보다 높은 k 설정)
    initial_results = vector_store.similarity_search(query_embedding, k=10)
    
    # reranking 적용
    if reranking_method == "llm":
        reranked_results = rerank_with_llm(query, initial_results, top_n=top_n)
    elif reranking_method == "keywords":
        reranked_results = rerank_with_keywords(query, initial_results, top_n=top_n)
    else:
        # reranking 없이 초기 검색 결과 사용
        reranked_results = initial_results[:top_n]
    
    # reranking 결과의 컨텍스트 결합
    context = "\n\n===\n\n".join([result["text"] for result in reranked_results])
    
    # 컨텍스트를 기반으로 응답 생성
    response = generate_response(query, context, model_name)
    
    return {
        "query": query,
        "reranking_method": reranking_method,
        "initial_results": initial_results[:top_n],
        "reranked_results": reranked_results,
        "context": context,
        "response": response
    }

## Reranking 품질 평가

In [ ]:
import pandas as pd

# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')

# 평가 데이터에서 첫 번째 쿼리 추출
query = df['query'][0]

# 참고(정답) 답변
reference_answer = df['generation_gt'][0]

# 파일 경로
file_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"


In [ ]:
# Process document
vector_store = process_document(file_path)

# 각각 방법 비교
print("Comparing retrieval methods...")

# 1. Standard retrieval (no reranking)
print("\n=== STANDARD RETRIEVAL ===")
standard_results = rag_with_reranking(query, vector_store, reranking_method="none")
print(f"\nQuery: {query}")
print(f"\nResponse:\n{standard_results['response']}")

# 2. LLM-based reranking
print("\n=== LLM-BASED RERANKING ===")
llm_results = rag_with_reranking(query, vector_store, reranking_method="llm")
print(f"\nQuery: {query}")
print(f"\nResponse:\n{llm_results['response']}")

# 3. Keyword-based reranking
print("\n=== KEYWORD-BASED RERANKING ===")
keyword_results = rag_with_reranking(query, vector_store, reranking_method="keywords")
print(f"\nQuery: {query}")
print(f"\nResponse:\n{keyword_results['response']}")

In [ ]:
def evaluate_reranking(query, standard_results, reranked_results, reference_answer=None):
    """
    reranking 결과와 기본 검색 결과를 비교하여 최종 생성된 응답 품질 평가
    
    Args:
        query (str): 사용자 쿼리
        standard_results (Dict): 기본 검색 결과
        reranked_results (Dict): 재정렬된 결과
        reference_answer (str, optional): 비교를 위한 기준 답변
        
    Returns:
        str: Evaluation output
    """
    # 시스템 프롬프트
    system_prompt = """You are an expert evaluator of RAG systems.
    Compare the retrieved contexts and responses from two different retrieval methods.
    Assess which one provides better context and a more accurate, comprehensive answer."""
    
    # truncated 된 컨텍스트와 응답을 포함한 비교 텍스트 준비
    comparison_text = f"""Query: {query}

Standard Retrieval Context:
{standard_results['context'][:1000]}... [truncated]

Standard Retrieval Answer:
{standard_results['response']}

Reranked Retrieval Context:
{reranked_results['context'][:1000]}... [truncated]

Reranked Retrieval Answer:
{reranked_results['response']}"""

    # 정답(참조) 답변이 제공된 경우 비교 텍스트에 포함
    if reference_answer:
        comparison_text += f"""
        
Reference Answer:
{reference_answer}"""

    # user 프롬프트
    user_prompt = f"""
{comparison_text}

Please evaluate which retrieval method provided:
1. More relevant context
2. More accurate answer
3. More comprehensive answer
4. Better overall performance

Provide a detailed analysis with specific examples.
"""
    
    # 응답 평가
    response = client_openai.chat.completions.create(
        model='gpt-4.1-mini',
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 평가 결과 반환
    return response.choices[0].message.content

In [ ]:
# true_answer과 비교하여 reranking 성능 평가
evaluation = evaluate_reranking(
    query=query,  # 사용자 쿼리
    standard_results=standard_results,  # 기본 검색 결과
    reranked_results=llm_results,  # LLM 기반 재정렬 결과
    reference_answer=reference_answer  # 비교를 위한 기준 답변
)

# 평가 결과 출력
print("\n=== EVALUATION RESULTS ===")
print(evaluation)